In [30]:
# Report creater
import pandas as pd
import matplotlib.pyplot as plt

# read in plotting sheet
# markdown template

md_template = pd.read_excel('Figure_Template.xlsx')
template_df = pd.DataFrame(md_template)

In [31]:
def fig_plotter(
    df,
    x_col,
    y_cols,
    x_title="Elapsed Seconds",
    y_title="",
    title="plot",
    exp_r=None,
    y_lim=None,
    directory=False,
    
):
    fig, ax = plt.subplots()

    exists = False

    if isinstance(exp_r, str):
        exp_r = eval(exp_r)
    else:
        exp_r = [-1000000, 1000000]

    for i, y in enumerate(y_cols):
        # only plot if in expected range...
        if y in df:
            exists = True
            if exp_r[0] < df[y].mean() < exp_r[1]:
                ax.plot(df[x_col], df[y])
            else:
                y_cols[i] = f"OOR - {y}"
                ax.plot(0, 0)
        else:
            pass
    
    if y_lim:
        ax.set_ylim(y_lim)

    ax.legend(y_cols)
    ax.set_title(title)
    ax.set_xlabel(x_title)
    ax.set_ylabel(y_title)
    ax.grid()

    if exists:
        figname = directory + title + ".png"
        fig.savefig(figname)
    else:
        print(title + " --- DNE")
        figname = "NA"
    plt.close(fig)
    return figname

In [32]:
# to create upon finishing a run...
# converting array to markdown format of categories, figures, and table of contents
# categories
# - plots in categories
test_folder = 'Recuperated_02'
df = pd.read_csv(test_folder+'/4_5_23_Run2.csv')
toc_txt = ''
body_txt = '\n\n'


# test calcs
avg_pt = df[["P_PLC_PT_Exit_1", "P_PLC_PT_Exit_2", "P_PLC_PT_Exit_3"]].mean(axis=1)
df["PR"] = (avg_pt + 101.325) / 101.325
df["fuel_gph"] = df["VF_PLC_Fuel"] * 60 / 3.78541
df["fuel_kgs"] = df["VF_PLC_Fuel"] * 0.817 / 60

# summary table?


for group in template_df['Group'].unique():
    # print(x)
    toc_txt+=f"- [{group}](#{group.lower()})\n"
    body_txt+=f"# {group}\n\n"
    body_txt+=f"| | | |\n"
    body_txt+=f"|---|---|---|\n"
    tc = 1

    for i, row in template_df[template_df['Group']==group].iterrows():
        
        # try to create figure
        try:
            fig_plotter(
                    df,
                    row["x"],
                    eval(row["y"]),
                    row["x_label"],
                    row["y_label"],
                    row["title"],
                    row["Expected_range"],
                    directory=test_folder+'/Figs/'
                )

            # include if exists...
            title = row["title"]
            # summary table later maybe...
            # rmean = df[eval(row["y"])].mean()
            # toc_txt+=f"\t- {title}: mean:{rmean} \n"
            # toc_txt+=f"\t- [{title}](#{title.lower()})\n"
            # body_txt+=f"## {title}\n"
            body_txt+=f"|![{title}](Figs/{title}.png)"
            tc+= 1
            if tc == 4: 
                tc=0
                body_txt+=f"|\n"
 
        except Exception as e:
            print(f'failed to create: {row["title"]} because of {e}')
    body_txt+=f"\n\n"
        
with open(f"{test_folder}/{test_folder}.md", "w") as file:
    file.write(toc_txt+body_txt)
